In [ ]:
!pip install python_speech_features

# Imports

In [2]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np
from collections import defaultdict

#Connect To Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Get Distance

In [4]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

# Get Neighbors

In [5]:
def getNeighbors(trainingSet, instance, k):
    distances = []

    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

# Nearest Neighbors

In [6]:
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

# Evaluation

In [7]:
def getAccuracy(dataset, predictions):
    correct = 0 
    for x in range (len(dataset)):
        if dataset[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(dataset)

# Feature Extraction

In [8]:
directory = "/content/drive/My Drive/ML/genres/"
f= open("my.dat" ,'wb')
i=0

for folder in os.listdir(directory):
    i+=1
    if i==11 :
        break 	
    for file in os.listdir(directory+folder):	
        (rate,sig) = wav.read(directory+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)

f.close()

# Train-Test Split

In [20]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break	

    for x in range(len(dataset)):
        if random.random() <split :			
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])	

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.85, trainingSet, testSet)

# Make Prediction

In [21]:
leng_test = len(testSet)

predictions_test = []
for x in range (leng_test):
    predictions_test.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 3))) 

accuracyTest = getAccuracy(testSet , predictions_test)
print(accuracyTest)

0.7465753424657534


# Predict Genre

In [30]:
results=defaultdict(int)

i=1
for folder in os.listdir('/content/drive/My Drive/ML/genres'):
    results[i]=folder
    i+=1

(rate,sig)=wav.read('/content/drive/MyDrive/ML/test_audios/Artcell_Tomake.wav')
mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature=(mean_matrix,covariance,0)

pred=nearestClass(getNeighbors(dataset ,feature , 3))

print(results[pred])

classical
